## Imports

In [9]:
import torch

## CNN

In [27]:
%run classification/resnet.py

In [28]:
labels = ['disease1', 'disease2']

cnn = Resnet50CNN(labels, freeze=True)

In [29]:
batch_size = 4
h = w = 512

images = torch.rand(4, 3, h, w)
images.size()

torch.Size([4, 3, 512, 512])

In [30]:
features = cnn.features(images)
features.size()

torch.Size([4, 2048, 16, 16])

In [ ]:
# batch_size, n_features, height, width = features.size()

## Decoder

In [21]:
%run report_generation/decoder_lstm.py

In [22]:
vocab_size = 1000
embedding_size = 200
hidden_size = 100

decoder = LSTMDecoder(vocab_size, embedding_size, hidden_size)
decoder

LSTMDecoder(
  (embeddings_table): Embedding(1000, 200, padding_idx=0)
  (lstm_cell): LSTMCell(200, 100)
  (W_vocab): Linear(in_features=100, out_features=1000, bias=True)
)

In [23]:
batch_size = 4
hidden_size = 100

initial_state = torch.rand(batch_size, hidden_size)
initial_state.size()

torch.Size([4, 100])

In [24]:
outputs = decoder(initial_state, 10)
words = outputs[0]
words.size()

torch.Size([4, 10, 1000])

## Encoder-Decoder

In [44]:
%run report_generation/cnn_to_seq.py

In [45]:
model = CNN2Seq(cnn, decoder)
# model

In [46]:
words, = model(images, 10)
words.size()

torch.Size([4, 10, 1000])